In [3]:
pip install future

     |████████████████████████████████| 829 kB 180 kB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=0acad5511a66e55bb3f2aeb191fa7827ca5541c832ad0942e7f077f8891bcbd1
  Stored in directory: /home/dm20/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import time
import numpy as np
import math
import heapq
from past.builtins import xrange

# Function

In [26]:
import numpy as np
import math
import heapq
from past.builtins import xrange
def finding_biker(class_ids, boxes, confidences):
    per = [index for index in range(len(class_ids)) if class_ids[index] == 0]
    #print('PERSON: ',per)
    bike = [index for index in range(len(class_ids)) if (class_ids[index] == 1) or (class_ids[index] == 3)]
    #print('bike: ',bike)
    per_flag = False
    bike_flag = False

    euclidean_dist = []

    if len(per) >= len(bike):
        for p in per:
            dist = [math.sqrt(((boxes[p][0] - boxes[b][0]) **2) + ((boxes[p][1] - boxes[b][1]) **2)) for b in bike]
            euclidean_dist.append(dist)
            iter_len = len(bike)
            per_flag = True
    else:
        #print('Bike highest')
        for b in bike:
            dist = [math.sqrt(((boxes[b][0] - boxes[p][0]) **2) + ((boxes[b][1] - boxes[p][1]) **2)) for p in per]
            euclidean_dist.append(dist)
            iter_len = len(per)
            bike_flag = True


    euclidean_dist = np.array(euclidean_dist)
    #print(euclidean_dist)
    _del = []

    for i in range(iter_len):
        #print('Changing index :',i)
        argmin = heapq.nsmallest(len(euclidean_dist[:,i]), xrange(len(euclidean_dist[:,i])),\
                                 key = euclidean_dist[:,i].__getitem__)
        #print(argmin)
        #print(euclidean_dist[:,i])
        
        if bike_flag:
            argmin_original_index = [bike[index] for index in argmin]
        else:
            argmin_original_index = [per[index]for index in argmin]
        #print('Original index : ',argmin)

        for index,ind in zip(argmin_original_index,argmin):
            #print(euclidean_dist[:,index])
            if class_ids[index] == 2:
                continue
                
            else:
                    #print(euclidean_dist[:,i])
                    #result.append(index)
                if per_flag:
                    #print('Person Flag')
                    if (boxes[i][3] > boxes[index][3]) and (euclidean_dist[:,i][ind] < 300):
                        #print(class_ids[index])
                        class_ids[index] = 2
                        #print(class_ids[index])
                        #print(' Selected dist: ',euclidean_dist[:,i][ind])
                        _del.append(bike[i])
                        #print(boxes[index])
                        #print(boxes[i])
                        boxes[index] = [boxes[index][0], boxes[index][1], boxes[index][2], boxes[index][3],\
                                       boxes[index][4], int((boxes[index][5]+boxes[i][5])/2)]
                        #print(boxes[index], '\n')

                    break
                else:
                    #print(boxes[i][1])
                    #print(boxes[index][1])
                    #print(index,'hello: ',euclidean_dist[:,i][ind])
                    if (boxes[i][3] < boxes[index][3]) and (euclidean_dist[:,i][ind] < 300):
                        #print('old:',class_ids[index])
                        #print(' Selected dist: ',euclidean_dist[:,i][ind])
                        class_ids[index] = 2
                        #print('new:',class_ids[index])
                        _del.append(per[i])
                        #print(boxes[index])
                        #print(boxes[i])
                        boxes[index] = [boxes[index][0], boxes[index][1], boxes[i][2], boxes[i][3],\
                                       boxes[i][4], int((boxes[index][5]+boxes[i][5])/2)]
                        
                        #print(boxes[index])

                    break
    #print('old:', class_ids)
    class_ids =  [i for j, i in enumerate(class_ids) if j not in _del]
    boxes = [i for j, i in enumerate(boxes) if j not in _del]
    #print(boxes)
    confidences = [i for j, i in enumerate(confidences) if j not in _del]
    #print(class_ids)

    return class_ids, boxes, confidences

def yolo3_object_detection(net, classes, test_folder = 'wholeData3', destination_folder = 'img'):
    layer_names = net.getLayerNames()
    outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    colors= np.random.uniform(0,255,size=(len(classes),3))
    #loading image
    import glob
    #cap=cv2.VideoCapture(0) #0 for 1st webcam
    font = cv2.FONT_HERSHEY_PLAIN
    starting_time= time.time()
    frame_id = 0
    save=0

    for img_link in glob.glob(test_folder + '/*'):
        #_,frame= cap.read() # 
        frame_id+=1
        frame = cv2.imread(img_link)
        height,width,channels = frame.shape
        #detecting objects
        blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
        net.setInput(blob)
        outs = net.forward(outputlayers)
        #print(outs[1])


        #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
        class_ids=[]
        confidences=[]
        boxes=[]
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                #print(class_id)
                confidence = scores[class_id]
                if confidence > 0.3:
                    #onject detected
                    center_x= int(detection[0]*width)
                    center_y= int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                    #rectangle co-ordinaters
                    x=int(center_x - w/2)
                    y=int(center_y - h/2)
                    #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                    boxes.append([center_x, center_y, x, y, w, h])
                    #boxes.append([x, y, w, h])#put all rectangle areas
                    confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                    class_ids.append(class_id) #name of the object tha was detected
        
        '''My area'''
        #print(class_ids, boxes, confidences)
        class_ids, boxes, confidences = finding_biker(class_ids, boxes, confidences)
        
        
        '''End area'''
        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

        for i in range(len(boxes)):
            if i in indexes:
                _, _, x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence= confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            

        elapsed_time = time.time() - starting_time
        fps=elapsed_time/frame_id
        print('Frame Per Second:' ,fps)
        print('\n')
        cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
    
        cv2.imshow("Image",frame)
        cv2.imwrite(destination_folder + '/img' + str(save) + '.jpg',frame)
        save+=1
        key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
        if key == 27: #esc key stops the process
            break;
    
    #cap.release()    
    cv2.destroyAllWindows()    

In [27]:
#Load YOLO
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") # Original yolov3
#net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [28]:
yolo3_object_detection( net, classes, 'wholeData3', 'img')

Frame Per Second: 0.9182031154632568


Frame Per Second: 0.9331655502319336


Frame Per Second: 0.9024168650309244


Frame Per Second: 0.8849436044692993


Frame Per Second: 0.8734460353851319


Frame Per Second: 0.8593769470850626


Frame Per Second: 0.8544872828892299


Frame Per Second: 0.8479215800762177


Frame Per Second: 0.8449334568447537


Frame Per Second: 0.8444314956665039


Frame Per Second: 0.8496694781563499


Frame Per Second: 0.8761220971743265


Frame Per Second: 0.8922887398646429


Frame Per Second: 0.8923641954149518


Frame Per Second: 0.8874087651570638


Frame Per Second: 0.8867266923189163


Frame Per Second: 0.8860105205984676


Frame Per Second: 0.8869562149047852


Frame Per Second: 0.8882663877386796




# Class 

In [28]:
class Yolov3:
    
    def yolo3_object_detection(self, net, classes, test_folder = 'wholeData3', destination_folder = 'img'):
        layer_names = net.getLayerNames()
        outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
        colors= np.random.uniform(0,255,size=(len(classes),3))
        #loading image
        import glob
        #cap=cv2.VideoCapture(0) #0 for 1st webcam
        font = cv2.FONT_HERSHEY_PLAIN
        starting_time= time.time()
        frame_id = 0
        save=0

        for img_link in glob.glob(test_folder + '/*'):
            #_,frame= cap.read() # 
            frame_id+=1
            frame = cv2.imread(img_link)
            height,width,channels = frame.shape
            #detecting objects
            blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
            net.setInput(blob)
            outs = net.forward(outputlayers)
            #print(outs[1])


            #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
            class_ids=[]
            confidences=[]
            boxes=[]
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.3:
                        #onject detected
                        center_x= int(detection[0]*width)
                        center_y= int(detection[1]*height)
                        w = int(detection[2]*width)
                        h = int(detection[3]*height)

                        #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                        #rectangle co-ordinaters
                        x=int(center_x - w/2)
                        y=int(center_y - h/2)
                        #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                        boxes.append([x,y,w,h]) #put all rectangle areas
                        confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                        class_ids.append(class_id) #name of the object tha was detected

            indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

            for i in range(len(boxes)):
                if i in indexes:
                    x,y,w,h = boxes[i]
                    label = str(classes[class_ids[i]])
                    confidence= confidences[i]
                    color = colors[class_ids[i]]
                    cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                    cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
                

            elapsed_time = time.time() - starting_time
            fps=elapsed_time/frame_id
            print(fps)
            cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
    
            cv2.imshow("Image",frame)
            cv2.imwrite(destination_folder + '/img' + str(save) + '.jpg',frame)
            save+=1
            key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
            if key == 27: #esc key stops the process
                break;
    
        #cap.release()    
        cv2.destroyAllWindows()    

In [30]:
new_obj = Yolov3()
new_obj.yolo3_object_detection(net, classes, 'wholeData3', 'img')

0.5905580520629883
0.7131557464599609
0.8255808353424072
0.8537926077842712
0.9233726024627685
0.9615087509155273
0.991828441619873
0.9617898762226105
0.9358527130550809
0.9119095087051392
0.8956245075572621
0.8808038632074991
0.8672595024108887
0.8717101301465716
0.8635299682617188
0.8675733357667923
0.8680569564594942
0.870157414012485
0.867958759006701
0.8603815197944641
